Brecha Salarial de Género en Argentina 
Este proyecto fue realizado por el grupo 15 constituido por Daniela Ferraro y Jorgelina Sigal. Se utilizó un conjunto de datos provenientes del Ministerio de Trabajo, Empleo y Seguridad Social que contiene información sobre las remuneraciones en Argentina, desagregadas por género, tipo de industria y fecha. El objetivo es analizar las brechas salariales de género y su evolución. 

Dataset: https://www.trabajo.gob.ar/estadisticas/oede/empleoyremuneracion.asp

Contenido del dataset:
Periodo: indica año y mes al que corresponde la remuneración.
Industria: nombre del sector o rama de actividad económica.
Género: Género del grupo de trabajadores al que pertenece la remuneración.
Mujer: Variable dummy (1 o 0) toma el valor 1 si es mujer, 0 si no. Útil para análisis estadístico, modelos y gráficos.
Varon: Variable dummy (1 o 0) toma el valor 1 si es varón, 0 si no. Útil para análisis estadístico, modelos y gráficos.
Remuneración: Remuneración promedio mensual para ese sector, género y período (en pesos argentinos).


Objetivo general:
Analizar la evolución de la remuneración media del empleo registrado en el sector privado en Argentina, según rama de actividad y sexo, en el período comprendido entre enero de 1996 y octubre de 2022, utilizando herramientas de data science , con el fin de identificar patrones, desigualdades y tendencias vinculadas a la brecha salarial de género en el mundo laboral.

Objetivos específicos:
Procesar y limpiar el conjunto de datos proporcionado mediante herramientas de data science, utilizando bibliotecas como pandas para su análisis exploratorio.
Describir la evolución temporal de la remuneración media del empleo registrado en el sector privado, diferenciando por sexo y rama de actividad (según el CIIU rev. 3 a dos dígitos).
Comparar las trayectorias salariales entre varones y mujeres en los distintos sectores de actividad, identificando las ramas con mayores y menores brechas de género.
Visualizar los resultados mediante gráficos que faciliten la comprensión de las desigualdades salariales en el período analizado.
Reflexionar sobre las implicancias sociales y económicas de las desigualdades de género en el mundo laboral formal, a partir de los hallazgos del análisis de datos.


In [2]:
import pandas as pd

In [3]:
# Como primer paso, se decidio partir desde un dataset reducido para poder tener un objetivo claro
remuneracion = pd.read_csv('Remuneraciones (1).csv',header=None)
# Luego se empezó a convertir el dataset de horizontal a vertical
# Se extrajeron los encabezados, porque tenia encabezados en varias filas
industria_header = remuneracion.iloc[0]
genero_header = remuneracion.iloc[1]
# Se separaron los datos del contenido real. Se eliminaron las dos primeras filas (que eran encabezados) y se reseteo el índice
remuneracion_data = remuneracion.iloc[2:].reset_index(drop=True)
# Se agruparon las columnas como tuplas
remuneracion_data.columns = pd.MultiIndex.from_arrays([industria_header, genero_header])
# Se pasaron las columnas de multiindex a texto simple (menos periodo)
remuneracion_data.columns = [
    "Periodo" if i == 0 else f"{col[0]}_{col[1]}" 
    for i, col in enumerate(remuneracion_data.columns)
]
# Se transformo de horizontal a vertical
remuneraciones = remuneracion_data.melt(id_vars='Periodo', var_name='Industria_Genero', value_name='Valor')
# Se separo la columna combinada para dejarlas individuales
remuneraciones[['Industria', 'Genero']] = remuneraciones['Industria_Genero'].str.split('_', expand=True)
# Se reordenaron las columnas finales
remuneraciones = remuneraciones[['Periodo', 'Industria', 'Genero', 'Valor']]
# Se guarda el nuevo archivo
remuneraciones.to_csv("Rem2.csv", index=False)